In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt



In [37]:

data = pd.read_csv(r"data\data_summarized_by_month_filled_hosp_MR.tsv", sep="\t")

data["year_month"] = pd.to_datetime(data["year"].astype(str) + "-" + data["month"].astype(str).str.zfill(2) + "-01")


data_prepared = data.groupby("year_month", as_index=False).agg(
    lethal_outcomes_perc = ("Летальные_Исходы", lambda x: x.sum(skipna=True) / data["Пролечены_с_ОНМК_Всего"].sum(skipna=True) * 100),
    ishemic_stroke_perc = ("ИИ_Ишемический_Инсульт", lambda x: x.sum(skipna=True) / data["Пролечены_с_ОНМК_Всего"].sum(skipna=True) * 100),
    gem_stroke_perc = ("ГИ_Геморрагический_Инсульт", lambda x: x.sum(skipna=True) / data["Пролечены_с_ОНМК_Всего"].sum(skipna=True) * 100),
    age60_perc = ("Старше_60_лет", lambda x: x.sum(skipna=True) / data["Пролечены_с_ОНМК_Всего"].sum(skipna=True) * 100),
    severeNIHSSS_perc = ("NIHSS_Больше_21", lambda x: x.sum(skipna=True) / data["Пролечены_с_ОНМК_Всего"].sum(skipna=True) * 100),
    ii_therapeutic_window_perc = ("ИИ_Терапевтическое_Окно", lambda x: x.sum(skipna=True) / data["ИИ_Ишемический_Инсульт"].sum(skipna=True) * 100),
    gi_therapeutic_window_perc = ("ГИ_Терапевтическое_Окно", lambda x: x.sum(skipna=True) / data["ГИ_Геморрагический_Инсульт"].sum(skipna=True) * 100),
    total_therapeutic_window_perc = ("Терапевтическое_Окно_4_5ч", lambda x: x.sum(skipna=True) / data["Пролечены_с_ОНМК_Всего"].sum(skipna=True) * 100),
    thromb_perc = ("Тромболизис", lambda x: x.sum(skipna=True) / data["ИИ_Ишемический_Инсульт"].sum(skipna=True) * 100),
    total_treated = ("Пролечены_с_ОНМК_Всего", "sum"),
    total_load = ("Суммарная_Загрузка_ОНМК", "sum")
)



data_prepared['covid'] = ((data_prepared['year_month'] >= "2020-03-01") & 
                          (data_prepared['year_month'] <= "2021-12-31")).astype(int)


print(data_prepared.head())


  year_month  lethal_outcomes_perc  ishemic_stroke_perc  gem_stroke_perc  \
0 2017-01-01              0.168787             0.711543         0.099139   
1 2017-02-01              0.194513             0.956881         0.131140   
2 2017-03-01              0.259770             1.183395         0.136159   
3 2017-04-01              0.181964             0.880330         0.110433   
4 2017-05-01              0.186356             0.872801         0.106041   

   age60_perc  severeNIHSSS_perc  ii_therapeutic_window_perc  \
0    0.624325           0.130512                    0.061753   
1    0.850840           0.108551                    0.210113   
2    1.034059           0.175062                    0.205594   
3    0.854604           0.099139                    0.173211   
4    0.736014           0.114198                    0.176976   

   gi_therapeutic_window_perc  total_therapeutic_window_perc  thromb_perc  \
0                    0.221898                       0.148709     0.024852   
1   

In [38]:


# Function to perform ADF Test
def adf_test(series, name=""):
    result = adfuller(series.dropna())
    print(f"{name}: ADF Statistic = {result[0]:.4f}, p-value = {result[1]:.4f}")
    if result[1] < 0.05:
        print(f"✅ {name} is stationary")
    else:
        print(f"⚠️ {name} is NOT stationary (Differencing Needed)")

# Apply ADF Test to all variables
for col in ["lethal_outcomes_perc", "ishemic_stroke_perc", "gem_stroke_perc", 
            "age60_perc", "severeNIHSSS_perc", "total_therapeutic_window_perc", 
            "thromb_perc", "total_treated", "total_load"]:
    adf_test(data_prepared[col], name=col)


lethal_outcomes_perc: ADF Statistic = -0.7057, p-value = 0.8453
⚠️ lethal_outcomes_perc is NOT stationary (Differencing Needed)
ishemic_stroke_perc: ADF Statistic = -0.6045, p-value = 0.8700
⚠️ ishemic_stroke_perc is NOT stationary (Differencing Needed)
gem_stroke_perc: ADF Statistic = -0.4695, p-value = 0.8978
⚠️ gem_stroke_perc is NOT stationary (Differencing Needed)
age60_perc: ADF Statistic = 0.3981, p-value = 0.9814
⚠️ age60_perc is NOT stationary (Differencing Needed)
severeNIHSSS_perc: ADF Statistic = -0.9621, p-value = 0.7668
⚠️ severeNIHSSS_perc is NOT stationary (Differencing Needed)
total_therapeutic_window_perc: ADF Statistic = -2.8510, p-value = 0.0514
⚠️ total_therapeutic_window_perc is NOT stationary (Differencing Needed)
thromb_perc: ADF Statistic = -5.1495, p-value = 0.0000
✅ thromb_perc is stationary
total_treated: ADF Statistic = -0.2668, p-value = 0.9301
⚠️ total_treated is NOT stationary (Differencing Needed)
total_load: ADF Statistic = 0.7036, p-value = 0.9899
⚠️ 

In [39]:

data_diff = data_prepared.diff().dropna()

print(data_diff.head())


  year_month  lethal_outcomes_perc  ishemic_stroke_perc  gem_stroke_perc  \
1    31 days              0.025726             0.245338         0.032001   
2    28 days              0.065256             0.226514         0.005020   
3    31 days             -0.077805            -0.303065        -0.025726   
4    30 days              0.004392            -0.007530        -0.004392   
5    31 days              0.003137             0.140552         0.011922   

   age60_perc  severeNIHSSS_perc  ii_therapeutic_window_perc  \
1    0.226514          -0.021961                    0.148359   
2    0.183219           0.066511                   -0.004519   
3   -0.179454          -0.075923                   -0.032383   
4   -0.118590           0.015059                    0.003765   
5    0.215847           0.029491                    0.030877   

   gi_therapeutic_window_perc  total_therapeutic_window_perc  thromb_perc  \
1                    0.274453                       0.104786     0.014309   
2   

# VARMAX - все переменные зависимые

In [7]:
#from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX


Y = data_prepared[['lethal_outcomes_perc', 'ishemic_stroke_perc', 
               'total_therapeutic_window_perc', 'total_treated']]


# model = VAR(Y)
# result = model.fit(maxlags=1)  # Order = 1 (VAR(1))


# print(result.summary())


# Fit VARMAX model as an approximation for VARIMA
model = VARMAX(Y, order=(1,2)) #2,0,2
result = model.fit(disp=False)


print(result.summary())





c:\Users\VictoriaZaitceva\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
c:\Users\VictoriaZaitceva\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                                                   Statespace Model Results                                                                  
Dep. Variable:     ['lethal_outcomes_perc', 'ishemic_stroke_perc', 'total_therapeutic_window_perc', 'total_treated']   No. Observations:                   96
Model:                                                                                                    VARMA(1,2)   Log Likelihood                 -55.062
                                                                                                         + intercept   AIC                            234.125
Date:                                                                                               Wed, 29 Jan 2025   BIC                            393.114
Time:                                                                                                       19:08:30   HQIC                           298.391
Sample:                                             

Equation for Total Treated Patients (total_treated)

L1.lethal_outcomes_perc: -71.7665 (p = 0.000) → Highly Significant

A 1% increase in lethal outcomes percentage reduces the total treated by ~72 patients.
This is statistically significant, suggesting that as lethal outcomes increase, fewer patients receive treatment.


L1.ishemic_stroke_perc: 719.2147 (p = 0.000) → Highly Significant

A 1% increase in ischemic stroke rates increases total treated by 719 patients.
This is a strong and statistically significant relationship, indicating that hospitals treat more patients when ischemic strokes increase.



The model suggests that ischemic stroke rates significantly influence the number of patients treated, but lethal outcomes percentage does not significantly predict other variables.

🔹 The presence of multicollinearity and a singular covariance matrix suggests that this model may not be reliable.

🔹 Many coefficients are not statistically significant (p > 0.05), except for total_treated, which is strongly influenced by ischemic stroke rates.

🔹 A more refined model, such as SARIMAX with selected predictors, might be more stable and interpretable.

# VARMAX - зависимые + экзогенные

In [34]:
Y = data_prepared[['lethal_outcomes_perc', 'total_treated']]

# exogenous variable 
X = data_prepared[['covid', 'ishemic_stroke_perc', 'total_therapeutic_window_perc', "age60_perc", "severeNIHSSS_perc"]]

In [35]:
from statsmodels.tsa.statespace.varmax import VARMAX
model = VARMAX(endog=Y, exog=X, order=(1,1))  # VARIMA(p=1, d=1, q=1)
result = model.fit(disp=False)
print(result.summary())


c:\Users\VictoriaZaitceva\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
c:\Users\VictoriaZaitceva\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                       Statespace Model Results                                      
Dep. Variable:     ['lethal_outcomes_perc', 'total_treated']   No. Observations:                   96
Model:                                           VARMAX(1,1)   Log Likelihood                -341.541
                                                 + intercept   AIC                            729.081
Date:                                       Wed, 29 Jan 2025   BIC                            788.061
Time:                                               19:45:02   HQIC                           752.922
Sample:                                                    0                                         
                                                        - 96                                         
Covariance Type:                                         opg                                         
Ljung-Box (L1) (Q):             0.53, 1.19   Jarque-Bera (JB):         29.83, 18.4

**Lethal Outcomes Percentage**

Severe NIHSSS percentage (p = 0.008) and ischemic stroke percentage (p = 0.021) have a significant positive effect on lethal outcomes.

Age 60+ percentage and COVID period show weak or no significant effect.

**Total Treated**

Lethal outcomes percentage from the previous period strongly predicts the number of treated patients (p < 0.001, coefficient = 219.54), suggesting that an increase in lethal cases results in a higher number of treated patients.

Past total treated (L1.total_treated) is also significant (p < 0.001) but with a much smaller effect compared to lethal outcomes.

COVID period had a strong negative effect on total treated patients (p = 0.016, coefficient = -39.21), suggesting that during COVID, fewer patients were treated.

Ischemic stroke percentage (p < 0.001) is the strongest predictor of total treated cases, followed by:

- Therapeutic window percentage (p < 0.001)
- Age 60+ percentage (p < 0.001)
- Severe NIHSSS percentage (p < 0.001)
- These findings suggest that stroke-related factors heavily influence hospital admissions.


# SARIMAX

In [48]:
import pmdarima as pm

Y = data_prepared[['lethal_outcomes_perc']]

# exogenous variable 
X = data_prepared[['covid', 'ishemic_stroke_perc', 'total_therapeutic_window_perc', "age60_perc", "severeNIHSSS_perc", 'total_treated']]

# Use auto_arima to find optimal (p,d,q)
auto_model = pm.auto_arima(Y, exogenous=X, seasonal=False, stepwise=True, trace=True)
print(auto_model.summary())


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=-326.784, Time=0.33 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=-32.964, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=-296.807, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=-124.009, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=-330.088, Time=0.08 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=-172.515, Time=0.19 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=-330.743, Time=0.10 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=-329.632, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=-321.007, Time=0.09 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=-332.363, Time=0.42 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=-315.974, Time=0.07 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=-321.513, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=-331.098, Time=0.29 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=-331.510, Time=0.25 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept 

In [49]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(Y, exog=X, order=(1,0,1), enforce_stationarity=False, enforce_invertibility=False)
# Change optimization method
result = model.fit(method='powell', disp=False)
print(result.summary())


                                SARIMAX Results                                 
Dep. Variable:     lethal_outcomes_perc   No. Observations:                   96
Model:                 SARIMAX(1, 0, 1)   Log Likelihood                 231.639
Date:                  Wed, 29 Jan 2025   AIC                           -445.277
Time:                          20:00:56   BIC                           -422.388
Sample:                               0   HQIC                          -436.032
                                   - 96                                         
Covariance Type:                    opg                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
covid                             0.0239      0.015      1.610      0.107      -0.005       0.053
ishemic_stroke_perc               0.1040      0.067      1

✅ severeNIHSSS_perc (stroke severity) is the strongest predictor of lethal outcomes (p = 0.009).

❌ covid, ishemic_stroke_perc, age60_perc, total_treated, and total_therapeutic_window_perc are NOT significant.

✅ MA(1) is highly significant, meaning past error terms influence current mortality rates.

## Log Transformation for normality 
since residuals are highly non-normal, try log transformation

In [50]:

data_prepared['lethal_outcomes_perc_log'] = np.log1p(data_prepared['lethal_outcomes_perc'])

# Fit SARIMAX with transformed Y
Y_log = data_prepared['lethal_outcomes_perc_log']
model = SARIMAX(Y_log, exog=X, order=(1,0,1), enforce_stationarity=False, enforce_invertibility=False)
result = model.fit(method='powell', disp=False)

print(result.summary())

                                  SARIMAX Results                                   
Dep. Variable:     lethal_outcomes_perc_log   No. Observations:                   96
Model:                     SARIMAX(1, 0, 1)   Log Likelihood                 251.730
Date:                      Wed, 29 Jan 2025   AIC                           -485.460
Time:                              20:03:31   BIC                           -462.570
Sample:                                   0   HQIC                          -476.214
                                       - 96                                         
Covariance Type:                        opg                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
covid                             0.0196      0.011      1.712      0.087      -0.003       0.042
ishemic_stroke_perc       

In [51]:
X_reduced = data_prepared[['severeNIHSSS_perc', 'ishemic_stroke_perc', 'covid']]  # Keep only significant predictors

model = SARIMAX(Y_log, exog=X_reduced, order=(1,0,1), enforce_stationarity=False, enforce_invertibility=False)
result = model.fit(method='powell', disp=False)

print(result.summary())


                                  SARIMAX Results                                   
Dep. Variable:     lethal_outcomes_perc_log   No. Observations:                   96
Model:                     SARIMAX(1, 0, 1)   Log Likelihood                 250.532
Date:                      Wed, 29 Jan 2025   AIC                           -489.064
Time:                              20:05:20   BIC                           -473.805
Sample:                                   0   HQIC                          -482.901
                                       - 96                                         
Covariance Type:                        opg                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
severeNIHSSS_perc       0.3279      0.089      3.695      0.000       0.154       0.502
ishemic_stroke_perc     0.1346      0.018      7.330    

✅ All exogenous predictors (severeNIHSSS_perc, ishemic_stroke_perc, and covid) are statistically significant!

✅ AR(1) and MA(1) terms are significant → Model captures time-series dynamics well.


Diagnotics


✅ No autocorrelation issues → Model correctly captures trends.

✅ Variance is stable → Model is reliable.

❌ Residuals still slightly skewed, but improved → Possible further refinements (e.g., Box-Cox transformation).

Fit and performance are ok (best model so far)